In [156]:
from bs4 import BeautifulSoup
import sys
import time
import os
import logging
import argparse
import requests
import codecs
import urllib
import os
import urllib.request
import json
import nltk
from nltk.corpus import stopwords

def get_reviews(theurl):
    reviewInfo = {}
    
    stop =  stop = set(stopwords.words('english'))
    
    main_page = urllib.request.urlopen(theurl)
    soup = BeautifulSoup(main_page, "html.parser")
    
    review_div = soup.findAll('div',{'itemprop':'review'})

    reviewCount = 1
    for i in review_div: # iterating through review_div 
        # get review star rating
        reviewStar = float(i.find('meta',{'itemprop':'ratingValue'}).get('content', None))

        # get review body text
        reviewBody = i.find('p',{'itemprop':'description'})
        for txt in reviewBody:
            if type(txt) != '<p>' and not str(txt).startswith('<p>'):
                reviewText = txt
                
                # Split review into sentences, remove stopwords, extract parts-of-speech tags
                # (opt. if lots of reviews) store each review into MongoDB db called 'Reviews'

                reviewWords = []
                sentences = nltk.sent_tokenize(reviewText.lower())
                
                for sentence in sentences:
                    tokens = nltk.word_tokenize(sentence)
                    text = [w for w in tokens if w not in stop]
                    tagged_text = nltk.pos_tag(text)
                
                for word, tag in tagged_text:
                    reviewWords.append({'word': word, 'pos': tag})
        
        reviewInfo[reviewCount] = {'review_stars' : reviewStar, \
                                   'review_text' : reviewText, \
                                   'review_words' : reviewWords} 
        reviewCount += 1
        
    ## TODO: ITERATE THROUGH ALL PAGES OF REVIEWS FOR RESTAURANT ##
        
    return reviewInfo

In [161]:
def lemmatize(reviewDict):
    # loop through the reviews
    # get nouns and group them by lemma
    reviewCorpus = {}
    lemmatizer = nltk.WordNetLemmatizer()

    for review_count, review_content in reviewDict.items():
        nouns = []
        words = [w for w in review_content['review_words'] if w['pos'] in ['NN','NNS']]
        
        for w in words:
            nouns.append(lemmatizer.lemmatize(w['word']))
            
        reviewCorpus[review_count] = {'review_stars' : review_content['review_stars'], \
                                      'review_text' : review_content['review_text'], \
                                      'review_nouns' : nouns} 
    
    return reviewCorpus

In [ ]:
# feed reviews to gensim LDA model

import gensim
from gensim.corpora import BleiCorpus
from gensim import corpora



In [163]:
# Test: First page of Pavement reviews
first_page = 'https://www.yelp.com/biz/pavement-coffeehouse-boston'
these_reviews = get_reviews(first_page)

#print(these_reviews)
#print(lemmatize(these_reviews))